<a href="https://colab.research.google.com/github/ugursirvermez/PyTorch_Education/blob/main/13_pytorch_reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kurulumlar

In [4]:
# Reinforcement Learning için ihtiyacımız olan kütüphaneler
!pip3 install gym
!pip3 install atari-py
!pip3 install box2d-py
!pip3 install torch
!pip3 install tensorflow-gpu #PyTorch yerine kullanılır.

  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
  Running setup.py clean for box2d-py
Failed to build box2d-py
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (box2d-py)
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with th

## DEEPQ NETWORK Sınıfını Oluşturma

In [12]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F #CNN ve Non_linear fonksiyonlar burada
import torch.optim as optim #Optimizer ve Loss fonksiyonları burada hatırlayalım.
import numpy as np

# DeepQ Learning Sınıfı Oluşturalım.
class DeepQNetwork(nn.Module):

	#Sınıf başlatıldığında neler olacak?
	def __init___(self, ALPHA): #ALPHA = Learning Rate
		super(DeepQNetwork, self).__init__()
		#Burada iki tane evrişimsel ağ kurulacak.
		self.conv1 = nn.Conv2d(1, 32, 8, stride=4, padding=1) # (self, in_channels, out_channels, kernel_size, stride)
		self.conv2 = nn.Conv2d(32, 64, 4, stride=3) #ikiye katlandı.
		self.conv3 = nn.Conv2d(64,128,3) #Out çıktı.
		self.fc1 = nn.Linear(128*19*8, 512) #Boyutların çıktısı
		self.fc2 = nn.Linear(512, 6)

		#Loss ve Optimizer
		self.optimizer = optim.RMSprop(self.parameters(), lr=ALPHA)
		self.loss = nn.MSELoss()

		#Device GPU ? CPU
		self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
		self.to(self.device)

	#Forward Fonksiyonu
	def forward(self, observation): #gözlemler yaparak doğru çıktıya ulaşma çabası
		observation = T.Tensor(observation).to(self.device) #Gözlemleri CNN için Tensör haline getiriyoruz.
		observation = observation.view(-1, 1, 185, 95)
		observation = F.relu(self.conv1(observation)) #ReLU işlemini hatırlayalım.
		observation = F.relu(self.conv2(observation))
		observation = F.relu(self.conv3(observation))
		observation = observation.view(-1,128*19*8)
		observation = F.relu(self.fc1(observation))

		actions = self.fc2(observation)

		return actions

## Agent Sınıfı -> Eğitilecek Birim

In [14]:
#Agent Sınıfı
class Agent(object):
	def __init__(self, gamma, epsilon, alpha, maxMemorySize, epsEnd=0.05,
							 replace=10000, actionSpace=[0,1,2,3,4,5]):
		#
		self.GAMME = gamma
		self.EPSILON =epsilon
		self.EPS_END =epsEnd
		self.actionSpace = actionSpace
		self.memSize = maxMemorySize
		self.steps = 0
		self.learn_step_counter = 0
		self.memory = []
		self.memCntr = 0
		self.replace_target_cnt = replace
		self.Q_eval = DeepQNetwork(alpha)
		self.Q_next = DeepQNetwork(alpha)

	#Dönüşümleri yani bilgiler arası geçişleri saklama
	def storeTransition(self, state, action, reward, state_):
		if self.memCntr < self.memSize:
			self.memory.append([state, action, reward, state_])
		else:
			self.memory[self.memCntr % self.memSize] = [state, action, reward, state_]
		self.memCntr +=1

	#Aksiyonları seçme
	def chooseAction(self, observation):
		rand = np.random.random()
		actions = self.Q_eval.forward(observation)
		if rand < 1 - self.EPSILON:
			action = T.argmax(actions[1]).item()
		else:
			action = np.random.choice(self.actionSpace)
		self.steps += 1
		return action

	#Öğrenme Fonksiyonu Geliştirme
	def learn(self, batch_size):
    #her bir öğrenme adımında mevcut değeri hedeflenen değer ile karşılaştırma.
		self.Q_eval.optimizer.zero_grad() # Optimizer kullanıyoruz ama eğrimiz sıfıra oluaşmalı
		if self.replace_target_cnt is not None and \
			self.learn_step_counter % self.replace_target_cnt == 0:
			self.Q_next.load_state_dict(self.Q_eval.state_dict())

		#Hafızadaki değeri ölçme
		if self.memCntr + batch_size < self.memSize:
			memStart = int(np.random.choice(range(self.memCntr)))
		else:
			memStart = int(np.random.choice(range(self.memCntr-batch_size-1)))

		#Tahmini yürütme ve sonraki hamleyi hazırlama
		miniBatch = self.memory[memStart:memStart+batch_size]
		memory = np.array(miniBatch)
		Qpred = self.Q_eval.forward(list(memory[:, 0][:])).to(self.Q_eval.device)
		Qnext = self.Q_next.forward(list(memory[:, 3][:])).to(self.Q_eval.device)

		#Eylem sonucu nasıl oldu? Ödül mü?
		maxA = T.argmax(Qnext, dim=1).to(self.Q_eval.device)
		rewards = T.Tensor(list(memory[:, 2][:])).to(self.Q_eval.device)
		Qtarget[:, maxA] = rewards + self.GAMMA*T.max(Qnext[1])

		#İşlem adımı
		if self.steps > 500:
			if self.EPSILON - 1e-4 > self.EPS_END:
				self.EPSILON -= 1e-4
			else:
				self.EPSILON = self.EPS_END

		los = self.Q_eval.loss(Qtarget, Qpred).to(self.Q_eval.device)
		loss.backward()
		self.Q_eval.optimizer.step().self.learn_step_counter += 1

##MainLoop Oluşturma